In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Prepare Data

In [ ]:
# Read in the data: multiple CSV file in directory 'data'
# Path: prepare_data.ipynb
df_tf = pd.read_csv('tf_issues/tf_issues_classified.csv_0.csv')
df_torch = pd.read_csv('torch_issues/torch_issues_classified.csv_0.csv')

In [111]:
# Concatenate the dataframes

for i in range(1, 4):
    df_tf = pd.concat([df_tf, pd.read_csv('tf_issues/tf_issues_classified.csv_' + str(i) + '.csv')])

for i in range(1, 3):
    df_torch = pd.concat([df_torch, pd.read_csv('torch_issues/torch_issues_classified.csv_' + str(i) + '.csv')])

In [112]:
df_torch_bug.columns

Index(['Issue Number', 'Issue Title', 'Issue Body', 'Time created',
       'Time closed', 'Number of Assignees', 'Number of Comments', 'Tags',
       'Is Bug'],
      dtype='object')

In [113]:
#Getting the datafrom xlsx file in a dataframe 
df_tf_bug = pd.read_excel('/home/ayman/Downloads/dataset.xlsx')
df_torch_bug = pd.read_excel('/home/ayman/Downloads/PyTorchBugDataset.xlsx', sheet_name='Dataset')

In [114]:
buggy_issues_torch = df_torch_bug["Issue #"].tolist()
buggy_issues_tf = df_tf_bug["issue"].tolist()

In [115]:
#remove all the issues that none or nan from both lists
buggy_issues_torch = [x for x in buggy_issues_torch if str(x) != 'nan' and str(x) != 'none']
buggy_issues_tf = [x for x in buggy_issues_tf if str(x) != 'nan'and str(x) != 'none']

In [116]:
df_torch.loc[df_torch['Issue Number'].isin(buggy_issues_torch), 'Is Bug'] = True

In [117]:
#change any issue in buggy_issues_tf to Is Bug feature to true 
df_tf.loc[df_tf['Issue Number'].isin(buggy_issues_tf), 'Is Bug'] = True

### Helper Functions needed

In [ ]:
import re

def get_type(string):
    # Use regular expression to find all occurrences of 'name="type:xyz"'
    matches = re.findall(r'name="([^"]+)', string)
    #convert the list to string
    matches = ' '.join(matches)
    return matches

In [121]:
def Gathering_GT_Data(df, buggy_issues):
    df_bug = df[df['Issue Number'].isin(buggy_issues)]
    #get the examples that are buggy 
    df_temp = df[df['Is Bug'] == True]
    if(len(df_bug) < 200):
        df_bug = pd.concat([df_bug, df_temp[~df_temp['Issue Number'].isin(buggy_issues_tf)].head(200 - len(df_bug))])
    #get the examples that are not buggy
    df_bug = pd.concat([df_bug, df[df['Is Bug'] == False ].head(len(df_bug))])
    df_bug['Tags'] = df_bug['Tags'].apply(get_type)
    df_bug['Issue Title'] = df_bug['Issue Title'] + ' ' + df_bug['Tags']
    return df_bug


### Prepare The Data for TensorFlow Model

In [60]:
# #get the issues that are buggy using buggy issues list
# df_tf_bug = df_tf[df_tf['Issue Number'].isin(buggy_issues_tf)]
# #get the examples that are buggy 
# df_temp = df_tf[df_tf['Is Bug'] == True]
# df_tf_bug = pd.concat([df_tf_bug, df_temp[~df_temp['Issue Number'].isin(buggy_issues_tf)].head(100)])
# # #add 171 issues from df_tf that are not buggy to df_tf_bug
# df_tf_bug = pd.concat([df_tf_bug, df_tf[df_tf['Is Bug'] == False ].head(171)])

In [125]:
df_tf_bug = Gathering_GT_Data(df_tf, buggy_issues_tf)

In [126]:
df_tf_bug['Is Bug'].value_counts()

Is Bug
True     200
False    200
Name: count, dtype: int64

In [83]:
prepared_data_tf = df_tf_bug[['Issue Title', 'Issue Body', 'Is Bug']]

In [84]:
prepared_data_tf

,Issue Title,Issue Body,Is Bug
69,TF 2.0: tf.stack can cause a segmentation faul...,**System information**\n- Have I written custo...,True
145,[2.0] tf.numpy_function logs deprecation warni...,**System information**\n- Have I written custo...,True
216,Compilation error of TensorFlow Lite on Windows,<em>Please make sure that this is a bug. As pe...,True
336,[Bug report] wrong container setting in OpsTes...,File: tensorflow/core/kernels/ops_testutil.h\n...,True
780,Dataset iterator is stalled indefinitely with ...,**System information**\n- Have I written custo...,True
...,...,...,...
371,Android GPU Delegate Error while using groups ...,**System information**\n- Android 13:\n- Tenso...,False
372,Converted(quantized) model of simple dense neu...,### 1. System information\n\n- OS Platform and...,False
373,import tensorflow error stat:awaiting response...,### Issue type\n\nBuild/Install\n\n### Have yo...,False
374,ImportError: undefined symbol after install st...,### Issue type\n\nBug\n\n### Have you reproduc...,False


In [85]:
#add it in a csv file
prepared_data_tf.to_csv('GT_tf_data.csv', index=False)

### Prepare The Data for PyTorch Model

In [1]:
# #get the issues that are buggy using buggy issues list
# df_torch_bug = df_torch[df_torch['Issue Number'].isin(buggy_issues_torch)]
# #get the examples that are buggy 
# df_temp = df_torch[df_torch['Is Bug'] == True]
# # #add 171 issues from df_tf that are not buggy to df_tf_bug
# df_torch_bug = pd.concat([df_torch_bug, df_torch[df_torch['Is Bug'] == False ].head(194)])

In [ ]:
df_torch_bug = Gathering_GT_Data(df_torch, buggy_issues_torch)

In [97]:
df_torch_bug

,Issue Number,Issue Title,Issue Body,Time created,Time closed,Number of Assignees,Number of Comments,Tags,Is Bug
7842,71280,torch dependencies aren't fully specified + pk...,### 🐛 Describe the bug\n\nIn pytorch/torch_ver...,2022-01-13 20:36:39+00:00,2022-01-18 22:12:32+00:00,1,13,"[Label(name=""triage review""), Label(name=""modu...",True
7874,71089,Segfault using `torch.unique` on tensor with N...,### 🐛 Describe the bug\n\n`torch.unique(torch....,2022-01-10 12:59:06+00:00,2022-01-21 02:36:19+00:00,1,4,"[Label(name=""high priority""), Label(name=""modu...",True
7885,71063,Noisy warning raised by 'default_collate',### 🐛 Describe the bug\n\nAfter upgrading pyto...,2022-01-09 06:02:21+00:00,2022-01-21 19:35:40+00:00,0,2,"[Label(name=""triaged""), Label(name=""module: da...",True
8423,67626,Get rid of the blocking call in RRefProxy,"RRefProxy (`rref.rpc_async`, `rref.rpc_sync`, ...",2021-11-01 17:14:37+00:00,2022-01-19 23:25:02+00:00,1,1,"[Label(name=""oncall: distributed""), Label(name...",True
8519,67239,channels_last significantly degrades accuracy,## 🐛 Bug\n\nMoving a simple ResNet model and C...,2021-10-26 03:08:50+00:00,2021-11-03 16:49:51+00:00,0,16,"[Label(name=""high priority""), Label(name=""modu...",True
...,...,...,...,...,...,...,...,...,...
401,110857,torchscript throws warnings on everything,```py\nimport torch\n\nscripted = torch.jit.sc...,2023-10-09 09:24:32+00:00,2023-10-11 09:48:26+00:00,0,5,"[Label(name=""oncall: jit"")]",False
404,110839,DISABLED test_pinned_memory_with_cudaregister ...,Platforms: rocm\n\nThis test was disabled beca...,2023-10-09 00:57:03+00:00,2023-10-09 15:41:44+00:00,0,3,"[Label(name=""module: cuda""), Label(name=""modul...",False
405,110838,DISABLED test_pinned_memory_with_cudaregister ...,Platforms: rocm\n\nThis test was disabled beca...,2023-10-09 00:57:01+00:00,2023-10-09 03:39:28+00:00,0,6,"[Label(name=""module: cuda""), Label(name=""modul...",False
408,110812,SeLU Activation Function Implementation In GRU...,The actual task is to replace the tanh_() at l...,2023-10-08 06:33:30+00:00,2023-10-10 14:52:16+00:00,0,2,[],False


In [98]:
df_torch_bug['Is Bug'].value_counts()

Is Bug
True     194
False    194
Name: count, dtype: int64

In [101]:
GT_torch_data = df_torch_bug[['Issue Title', 'Issue Body', 'Is Bug']]

In [102]:
#to csv file
GT_torch_data.to_csv('GT_torch_data.csv', index=False)